In [130]:
import sys
from pathlib import Path
import importlib

ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

import src.topic_modeling as mod

importlib.reload(mod)

import pandas as pd
from src.topic_modeling import (
    print_topics, make_topic_labels, topic_top_words, strip_reddit_preview,
    nmf_topics
)

In [131]:
import sqlite3

PROCESSED_DIR = Path(ROOT) / "data" / "processed"
DB_CLEANED_PATH = PROCESSED_DIR / "sims4_cleaned.db"

print("DB path:", DB_CLEANED_PATH)
print("Exists?", DB_CLEANED_PATH.exists())

conn = sqlite3.connect(DB_CLEANED_PATH)

df = pd.read_sql_query("SELECT * FROM sarcasm_sentiment_per_post", conn)
display(df.head(5))

DB path: /Users/baderrezek/Desktop/Projects/Personal/sims4-sentiment-analysis/data/processed/sims4_cleaned.db
Exists? True


,negative_prob,neutral_prob,positive_prob,sarcasm_prob,negative_raw,neg_w_sarcasm,full_post,date,post_id
0,0.221770,0.611407,0.166823,0.071126,0.054947,0.059519,the cat is under the covers with her are you k...,2025-08-14 20:14:34+00:00,1mqbu8g
1,0.025228,0.075202,0.899570,0.048953,-0.874342,-0.841625,Welp. My 3 year old turned off my computer. Lo...,2025-08-20 03:09:47+00:00,1mv3jnz
2,0.007088,0.032331,0.960581,0.103234,-0.953493,-0.879188,I love everything about this ridiculously big ...,2025-08-18 20:20:44+00:00,1mtxbiy
3,0.355312,0.527494,0.117194,0.050018,0.238118,0.235379,Does this look like a strawberry roll shortcak...,2025-08-28 19:01:04+00:00,1n2l1np
4,0.020656,0.222478,0.756866,0.046406,-0.736209,-0.709362,I got tired of tracking down fruits and veg so...,2025-08-11 10:36:01+00:00,1mn9ghp


In [134]:
complaints = df.query("neg_w_sarcasm > 0.2")

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(complaints["full_post"].tolist(), batch_size=256, show_progress_bar=True)

Batches: 100%|██████████| 11/11 [00:03<00:00,  2.94it/s]


In [140]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(complaints["full_post"].tolist(), embeddings)

# Inspect results
topic_model.get_topic_info().head(10)

2025-09-04 02:28:08,927 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-04 02:28:13,485 - BERTopic - Dimensionality - Completed ✓
2025-09-04 02:28:13,486 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-04 02:28:13,677 - BERTopic - Cluster - Completed ✓
2025-09-04 02:28:13,679 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-04 02:28:13,749 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,920,-1_to_the_and_in,"[to, the, and, in, my, of, is, sims, sim, this]",[How To Create A Family That Will Tear Itself ...
1,0,289,0_this_what_is_me,"[this, what, is, me, why, you, know, excuse, s...",[What is this in his head?:\nIve never seen th...
2,1,272,1_mods_game_it_my,"[mods, game, it, my, the, to, and, update, but...",[My builds keep getting deleted:\nSo my game b...
3,2,128,2_baby_toddler_infant_is,"[baby, toddler, infant, is, up, the, my, to, a...","[newborn bug:\nas depicted, i’m having a bug w..."
4,3,127,3_sims_the_of_to,"[sims, the, of, to, that, game, for, be, and, in]","[I don’t want sims 5, i want a sims 2 remake. ..."
5,4,124,4_build_house_the_it,"[build, house, the, it, roof, building, to, im...",[Few Beginner Questions (Mostly About Building...
6,5,70,5_sim_my_died_this,"[sim, my, died, this, is, male, at, time, just...",[What is the creepiest thing that's happened i...
7,6,58,6_she_her_shes_is,"[she, her, shes, is, bitch, why, moyer, alex, ...","[Why did she paint snow?:\nAt first, she start..."
8,7,51,7_save_files_saves_gone,"[save, files, saves, gone, my, lost, all, to, ...",[Computer broke down and I lost 8 years of gam...
9,8,49,8_twins_and_baby_to,"[twins, and, baby, to, the, of, they, family, ...",[Uuuhhhhh Twins?:\nSo I'm very new to the sims...


In [141]:

new_labels = {
    -1: "Bugs, Annoyances & Missing Features",
    0: "Mod/CC Conflicts & Patch Breakage",
    1: "Build/Buy Mode Confusion & Limitations",
    2: "N/A",
    3: "Weird or Unintended Gameplay Events",
    4: "Sims 5 Critiques & Franchise Uncertainty",
    5: "Save File Loss & Recovery Issues",
    6: "Core Gameplay Failures (Relationships, Weddings, Traits)",
    7: "CAS Representation & Customization Gaps",
    8: "Family & Child Gameplay Issues"
}

topic_model.set_topic_labels(new_labels)

In [142]:
display(topic_model.get_topic_info().head(10))

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,920,-1_to_the_and_in,"Bugs, Annoyances & Missing Features","[to, the, and, in, my, of, is, sims, sim, this]",[How To Create A Family That Will Tear Itself ...
1,0,289,0_this_what_is_me,Mod/CC Conflicts & Patch Breakage,"[this, what, is, me, why, you, know, excuse, s...",[What is this in his head?:\nIve never seen th...
2,1,272,1_mods_game_it_my,Build/Buy Mode Confusion & Limitations,"[mods, game, it, my, the, to, and, update, but...",[My builds keep getting deleted:\nSo my game b...
3,2,128,2_baby_toddler_infant_is,N/A,"[baby, toddler, infant, is, up, the, my, to, a...","[newborn bug:\nas depicted, i’m having a bug w..."
4,3,127,3_sims_the_of_to,Weird or Unintended Gameplay Events,"[sims, the, of, to, that, game, for, be, and, in]","[I don’t want sims 5, i want a sims 2 remake. ..."
5,4,124,4_build_house_the_it,Sims 5 Critiques & Franchise Uncertainty,"[build, house, the, it, roof, building, to, im...",[Few Beginner Questions (Mostly About Building...
6,5,70,5_sim_my_died_this,Save File Loss & Recovery Issues,"[sim, my, died, this, is, male, at, time, just...",[What is the creepiest thing that's happened i...
7,6,58,6_she_her_shes_is,"Core Gameplay Failures (Relationships, Wedding...","[she, her, shes, is, bitch, why, moyer, alex, ...","[Why did she paint snow?:\nAt first, she start..."
8,7,51,7_save_files_saves_gone,CAS Representation & Customization Gaps,"[save, files, saves, gone, my, lost, all, to, ...",[Computer broke down and I lost 8 years of gam...
9,8,49,8_twins_and_baby_to,Family & Child Gameplay Issues,"[twins, and, baby, to, the, of, they, family, ...",[Uuuhhhhh Twins?:\nSo I'm very new to the sims...
